In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import Wav2Vec2Model

class SpeakerSeparationModel(nn.Module):
    def __init__(self, num_speakers):
        super(SpeakerSeparationModel, self).__init__()
        
        # Frozen Wav2Vec2 base
        self.wav2vec2 = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h")
        for param in self.wav2vec2.parameters():
            param.requires_grad = False
        
        # Learnable clustering head
        self.clustering_head = nn.Linear(self.wav2vec2.config.hidden_size, num_speakers * self.wav2vec2.config.hidden_size)
        
        # Learnable linear layer for conversion
        self.linear_conversion = nn.Linear(self.wav2vec2.config.hidden_size * num_speakers, 2 * self.wav2vec2.config.hidden_size)
        
    def forward(self, input_waveform, labels=None):
        # Wav2Vec2 forward pass
        #print(input_waveform.flatten(1).shape)
        print(self.wav2vec2.config)
        wav2vec2_output = self.wav2vec2(input_waveform.flatten(1)).last_hidden_state
        print(wav2vec2_output.reshape(-1,self.wav2vec2.config.hidden_size).shape)
        # Learnable clustering head forward pass
        clustering_output = self.clustering_head(wav2vec2_output.flatten(1))
        
        # Learnable linear layer for conversion
        conversion_output = self.linear_conversion(clustering_output)
        
        # Determine the size based on the input waveform size
        batch_size, _, amplitude = input_waveform.size()
        conversion_output = conversion_output.view(batch_size, 2, -1)
        
        return conversion_output

# Define your test input waveform
test_input_waveform = torch.randn(1, 1, 739440)  # Adjusted to the provided amplitude value

# Initialize the SpeakerSeparationModel
num_speakers = 2
model = SpeakerSeparationModel(num_speakers)

# Forward pass to get the output
with torch.no_grad():
    model.eval()
    #print(test_input_waveform.shape)
    output = model(test_input_waveform)

# Print or use the output as needed
print("Model Output Shape:", output.shape)
print("Model Output:", output)


Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Wav2Vec2Config {
  "_name_or_path": "facebook/wav2vec2-base-960h",
  "activation_dropout": 0.1,
  "adapter_attn_dim": null,
  "adapter_kernel_size": 3,
  "adapter_stride": 2,
  "add_adapter": false,
  "apply_spec_augment": true,
  "architectures": [
    "Wav2Vec2ForCTC"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 1,
  "classifier_proj_size": 256,
  "codevector_dim": 256,
  "contrastive_logits_temperature": 0.1,
  "conv_bias": false,
  "conv_dim": [
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "conv_kernel": [
    10,
    3,
    3,
    3,
    3,
    2,
    2
  ],
  "conv_stride": [
    5,
    2,
    2,
    2,
    2,
    2,
    2
  ],
  "ctc_loss_reduction": "sum",
  "ctc_zero_infinity": false,
  "diversity_loss_weight": 0.1,
  "do_stable_layer_norm": false,
  "eos_token_id": 2,
  "feat_extract_activation": "gelu",
  "feat_extract_dropout": 0.0,
  "feat_extract_norm": "group",
  "feat_proj_dropout": 0.1,
  "feat_quantizer_dropout": 0.0,
  "final_dropout": 0

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x1774080 and 768x1536)

In [21]:
import torch

# Define your test input waveform
test_input_waveform = torch.randn(1, 732440)  # Replace 'amplitude' with the actual size of your input waveform

# Initialize the SpeakerSeparationModel
num_speakers = 2  # Adjust as per your requirement
model = SpeakerSeparationModel(num_speakers)

# Forward pass to get the output
with torch.no_grad():
    model.eval()  # Set the model to evaluation mode
    output = model(test_input_waveform).logits

# Print or use the output as needed
print("Model Output Shape:", output.shape)
print("Model Output:", output)


Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


torch.Size([1, 2288, 768])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x1757184 and 768x1536)

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import Wav2Vec2Model

class SpeakerSeparationModel(nn.Module):
    def __init__(self, num_speakers):
        super(SpeakerSeparationModel, self).__init__()
        
        # Frozen Wav2Vec2 base
        self.wav2vec2 = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h")
        for param in self.wav2vec2.parameters():
            param.requires_grad = False
        
        # Learnable clustering head
        self.clustering_head = nn.Linear(self.wav2vec2.config.hidden_size, num_speakers * self.wav2vec2.config.hidden_size)
        
        # Learnable linear layer for conversion
        self.linear_conversion = nn.Linear(self.wav2vec2.config.hidden_size * num_speakers, 2 * 739440)  # Adjusted the output size
        
    def forward(self, input_waveform, labels=None):
        # Wav2Vec2 forward pass
        wav2vec2_output = self.wav2vec2(input_waveform).last_hidden_state
        
        # Learnable clustering head forward pass
        clustering_output = self.clustering_head(wav2vec2_output.flatten(1))
        
        # Learnable linear layer for conversion
        conversion_output = self.linear_conversion(clustering_output)
        
        return conversion_output.view(conversion_output.size(0), 2, -1)

# Define your test input waveform
test_input_waveform = torch.randn(1, 739440)  # Adjusted to the provided amplitude value

# Initialize the SpeakerSeparationModel
num_speakers = 2
model = SpeakerSeparationModel(num_speakers)

# Forward pass to get the output
with torch.no_grad():
    model.eval()
    output = model(test_input_waveform)

# Print or use the output as needed
print("Model Output Shape:", output.shape)
print("Model Output:", output)


Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x1774080 and 768x1536)

In [1]:
from utils import LibriMixDataset


In [2]:
train_set, val_set = LibriMixDataset(dev_mode=True).load_dataset()

In [3]:
train_set = train_set[:50]

In [6]:
import torch
from torch.nn.functional import pad

def custom_collate_fn(batch):
    # Extract individual components from the batch
    sampling_rates, mixture_waveforms, source_waveforms = zip(*batch)

    # Find the maximum length of the mixture waveforms
    max_mixture_length = max(waveform.shape[1] for waveform in mixture_waveforms)

    # Pad the mixture waveforms
    padded_mixture_waveforms = torch.stack([
        pad(waveform, (0, max_mixture_length - waveform.shape[1]))
        for waveform in mixture_waveforms
    ], dim=0)

    # Find the maximum length of the source waveforms
    max_source_length = max(waveform.shape[1] for sources in source_waveforms for waveform in sources)

    # Pad the source waveforms
    padded_source_waveforms = [
        torch.stack([
            pad(waveform, (0, max_source_length - waveform.shape[1]))
            for waveform in sources
        ], dim=0)
        for sources in zip(*source_waveforms)
    ]

    return {
        'sampling_rates': torch.tensor(sampling_rates),
        'mixture_waveforms': padded_mixture_waveforms,
        'source_waveforms': padded_source_waveforms
    }


In [15]:
import torch
from torch import nn
from sklearn.cluster import KMeans
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

# Define Wav2Vec 2.0 model and processor
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

# Freeze the parameters of the Wav2Vec 2.0 model
for param in model.parameters():
    param.requires_grad = False

# Define your clustering model
num_clusters = 2  # Use the number of source waveforms as clusters
kmeans = KMeans(n_clusters=num_clusters, random_state=0)

# Define your training data
# Replace this with your actual training data loading logic
sampling_rate = 16000  # Adjust based on your data
#train_set = [(sampling_rate, mix_waveform, source_waveforms)]

# Feature extraction and clustering
for _, mix_waveform, source_waveforms in train_set:
    # Feature extraction using Wav2Vec 2.0
    mix_input_values = processor(mix_waveform, return_tensors="pt", padding="longest", sampling_rate=16000).input_values
    mix_input_values = mix_input_values.squeeze()  # Remove unnecessary dimensions
    mix_outputs = model(mix_input_values.unsqueeze(0))  # Add batch dimension
    mix_logits = mix_outputs.logits.squeeze()
    #print(mix_logits.shape)

    source_features_list = []
    for source_waveform in source_waveforms:
        source_input_values = processor(source_waveform, return_tensors="pt", padding="longest", sampling_rate=16000).input_values
        source_input_values = source_input_values.squeeze()  # Remove unnecessary dimensions
        source_outputs = model(source_input_values.unsqueeze(0))  # Add batch dimension
        source_logits = source_outputs.logits.squeeze()
        source_features_list.append(source_logits)
        #print(len(source_features_list))

    # Convert features to numpy arrays
    mix_features_np = mix_logits.detach().numpy()
    #print(mix_features_np.shape)
    source_features_np = torch.cat(source_features_list).detach().numpy()
    #print(source_features_np.shape)
    # Apply KMeans clustering to mix features
    mix_cluster_labels = kmeans.fit_predict(mix_features_np)

    # Separate mix waveform based on cluster labels
    
    from sklearn.metrics import davies_bouldin_score

    davies_bouldin = davies_bouldin_score(mix_features_np, mix_cluster_labels)
    print(f"Davies-Bouldin Index: {davies_bouldin}")

    # Apply KMeans clustering to source features
    # Extract cluster information
    # cluster_centers = torch.tensor(kmeans.cluster_centers_)  # Centroids of clusters
    # cluster_labels = torch.tensor(mix_cluster_labels)  # Labels assigned to each point
    # 
    # # Example: Access the cluster information for the first cluster
    # cluster_center = cluster_centers[0]
    # cluster_points = mix_logits[cluster_labels == 0]
    # Separate mix waveform based on cluster labels
    # separated_waveforms = []
    # for cluster_label in range(num_clusters):
    #     cluster_indices = torch.nonzero(cluster_label == mix_cluster_labels)
    #     cluster_waveform = mix_waveform[:, cluster_indices].mean(dim=1)  # You can use mean or any other aggregation method
    #     separated_waveforms.append(cluster_waveform)

    # 'separated_waveforms' now contains individual source waveforms

# You can further process or save the separated_waveforms as needed


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\Kedar Pandya\anaconda3\envs\friday\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Davies-Bouldin Index: 1.8841861514084979


C:\Users\Kedar Pandya\anaconda3\envs\friday\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Davies-Bouldin Index: 2.1304139086878573


C:\Users\Kedar Pandya\anaconda3\envs\friday\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


...

C:\Users\Kedar Pandya\anaconda3\envs\friday\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Davies-Bouldin Index: 1.674527619585569


C:\Users\Kedar Pandya\anaconda3\envs\friday\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Davies-Bouldin Index: 1.8888159301456398
Davies-Bouldin Index: 2.124545250811051


C:\Users\Kedar Pandya\anaconda3\envs\friday\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [30]:
import torch
from torch import nn
from sklearn.cluster import KMeans, DBSCAN, SpectralClustering
from sklearn.metrics import davies_bouldin_score
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

# Define Wav2Vec 2.0 model and processor
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

# Freeze the parameters of the Wav2Vec 2.0 model
for param in model.parameters():
    param.requires_grad = False

# Define your clustering model
num_clusters = 2  # Use the number of source waveforms as clusters
kmeans = KMeans(n_clusters=num_clusters, random_state=0)
#dbscan = DBSCAN(eps=0.5, min_samples=2)
spectral_clustering = SpectralClustering(n_clusters=num_clusters, random_state=0)

# Define your training data
# Replace this with your actual training data loading logic
sampling_rate = 16000  # Adjust based on your data

# Feature extraction and clustering
mix_features_list = []
source_features_list = []

for _, mix_waveform, source_waveforms in train_set:
    # Feature extraction using Wav2Vec 2.0 for mix waveform
    mix_input_values = processor(mix_waveform, return_tensors="pt", padding="longest", sampling_rate=16000).input_values.squeeze()
    mix_outputs = model(mix_input_values.unsqueeze(0))
    mix_logits = mix_outputs.logits.squeeze()
    mix_features_list.append(mix_logits.detach())

    # Feature extraction using Wav2Vec 2.0 for each source waveform
    for source_waveform in source_waveforms:
        source_input_values = processor(source_waveform, return_tensors="pt", padding="longest", sampling_rate=16000).input_values.squeeze()
        source_outputs = model(source_input_values.unsqueeze(0))
        source_logits = source_outputs.logits.squeeze()
        source_features_list.append(source_logits.detach())

# Convert features to numpy arrays
mix_features_np = torch.cat(mix_features_list).detach().numpy()
source_features_np = torch.cat(source_features_list).detach().numpy()

# Apply KMeans clustering to mix features
mix_cluster_labels_k = kmeans.fit_predict(mix_features_np)
mix_cluster_labels_s = spectral_clustering.fit_predict(mix_features_np)
# Separate mix waveform based on cluster labels

# Calculate Davies-Bouldin Index
davies_bouldin_k = davies_bouldin_score(mix_features_np, mix_cluster_labels_k)
davies_bouldin_s = davies_bouldin_score(mix_features_np, mix_cluster_labels_s)
print(f"Davies-Bouldin Index for K-Means: {davies_bouldin_k}")
print(f"Davies-Bouldin Index for Spectral Clustering: {davies_bouldin_s}")

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\Kedar Pandya\anaconda3\envs\friday\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


KeyboardInterrupt: 

In [31]:
rint(mix_cluster_labels.shape)

(228,)


In [41]:
from torch.utils.data import DataLoader
train_loader = DataLoader(train_set, batch_size=2, shuffle=True, collate_fn= custom_collate_fn)


In [26]:
import torch
import torch.nn as nn
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score
from torch.utils.data import DataLoader
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
from tqdm import tqdm

# Define Wav2Vec 2.0 model and processor
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

# Freeze the parameters of the Wav2Vec 2.0 model
for param in model.parameters():
    param.requires_grad = False

# Define your learnable clustering model
class ClusteringModel(nn.Module):
    def __init__(self, input_dim, num_clusters):
        super(ClusteringModel, self).__init__()
        self.fc = nn.Linear(input_dim, num_clusters)

    def forward(self, x):
        return self.fc(x)

# Define your DataLoader
# Adjust the batch size and collate function based on your requirements
train_loader = DataLoader(train_set, batch_size=2, shuffle=True, collate_fn=custom_collate_fn)

# Feature extraction and clustering
for batch in train_loader:
    # Unpack the batch
    sampling_rate, mix_waveforms, source_waveforms_list = batch

    # Feature extraction using Wav2Vec 2.0
    mix_input_values = processor(mix_waveforms, return_tensors="pt", padding="longest").input_values
    mix_input_values = mix_input_values.squeeze()  # Remove unnecessary dimensions
    mix_outputs = model(mix_input_values.unsqueeze(0))  # Add batch dimension
    mix_logits = mix_outputs.logits.squeeze()

    source_features_list = []
    for source_waveforms in source_waveforms_list:
        source_input_values = processor(source_waveforms, return_tensors="pt", padding="longest").input_values
        source_input_values = source_input_values.squeeze()  # Remove unnecessary dimensions
        source_outputs = model(source_input_values.unsqueeze(0))  # Add batch dimension
        source_logits = source_outputs.logits.squeeze()
        source_features_list.append(source_logits)

    # Convert features to PyTorch tensors
    mix_features = mix_logits
    source_features = torch.cat(source_features_list)

    # Define your learnable clustering model
    input_dim = mix_features.size(1)
    num_clusters = len(source_waveforms)  # Use the number of sources as clusters
    clustering_model = ClusteringModel(input_dim, num_clusters)

    # Define your loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(clustering_model.parameters(), lr=0.001)

    # Training loop with tqdm
    num_epochs = 10  # Adjust as needed
    with tqdm(total=num_epochs, desc="Training Clustering Model") as pbar:
        for epoch in range(num_epochs):
            # Forward pass
            cluster_assignments = clustering_model(mix_features)

            # Assuming you have ground truth cluster labels, replace 'source_labels' accordingly
            source_labels = torch.cat([torch.full((source.size(1),), i) for i, source in enumerate(source_features_list)])

            # Compute the loss
            loss = criterion(cluster_assignments, source_labels)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            pbar.update(1)
            pbar.set_postfix({"Loss": loss.item()})

    # After training, you can use clustering_model to predict cluster assignments for new data
    new_data_features = torch.tensor(val_set[0])
    predicted_labels = torch.argmax(clustering_model(new_data_features), dim=1)

    # Evaluate clustering performance using adjusted_rand_score and normalized_mutual_info_score
    ari_score = adjusted_rand_score(source_labels.numpy(), predicted_labels.numpy())
    nmi_score = normalized_mutual_info_score(source_labels.numpy(), predicted_labels.numpy())

    print(f"Adjusted Rand Index: {ari_score}")
    print(f"Normalized Mutual Information: {nmi_score}")


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


ValueError: type of mixture_waveforms unknown: <class 'str'>. Should be one of a python, numpy, pytorch or tensorflow object.

In [14]:
import torch
from torch import nn
from sklearn.cluster import KMeans
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

# Define Wav2Vec 2.0 model and processor
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

# Define your clustering model
num_clusters = len(source_waveforms)  # Use the number of source waveforms as clusters
kmeans = KMeans(n_clusters=num_clusters, random_state=0)

# Define your training data
# Replace this with your actual training data loading logic
sampling_rate = 16000  # Adjust based on your data
#train_set = [(sampling_rate, mix_waveform, source_waveforms)]

# Feature extraction and clustering
for _, mix_waveform, source_waveforms in train_set:
    # Feature extraction using Wav2Vec 2.0
    mix_input_values = processor(mix_waveform, return_tensors="pt", padding="longest").input_values
    mix_features = model(mix_input_values.squeeze())

    source_features_list = []
    for source_waveform in source_waveforms:
        source_input_values = processor(source_waveform, return_tensors="pt", padding="longest").input_values
        source_features = model(source_input_values.squeeze())
        source_features_list.append(source_features)

    # Convert features to numpy arrays
    mix_features_np = mix_features.detach().numpy()
    source_features_np = torch.cat(source_features_list).detach().numpy()

    # Apply KMeans clustering to mix features
    mix_cluster_labels = kmeans.fit_predict(mix_features_np)

    # Separate mix waveform based on cluster labels
    # separated_waveforms = []
    # for cluster_label in range(num_clusters):
    #     cluster_indices = (mix_cluster_labels == cluster_label)
    #     cluster_waveform = mix_waveform[:, cluster_indices].mean(dim=1)  # You can use mean or any other aggregation method
    #     separated_waveforms.append(cluster_waveform)

    # 'separated_waveforms' now contains individual source waveforms

# You can further process or save the separated_waveforms as needed


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


RuntimeError: Given groups=1, weight of size [512, 1, 10], expected input[1, 73040, 1] to have 1 channels, but got 73040 channels instead

In [16]:
from utils import LibriMixDataset
train_set, val_set = LibriMixDataset(dev_mode=True).load_dataset()
train_set = train_set[:50]

In [25]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from transformers import Wav2Vec2Processor, Wav2Vec2Model

# Assuming you have a pre-trained Wav2Vec2 model and processor
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h")
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")

# Define your source separation model
class SourceSeparationModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SourceSeparationModel, self).__init__()
        # Define your model architecture (e.g., an LSTM-based model)
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.linear = nn.Linear(hidden_size, output_size)

    def forward(self, mixture_embedding):
        # Pass through the LSTM and linear layer
        output, _ = self.lstm(mixture_embedding)
        separated_waveform = self.linear(output)

        return separated_waveform

# Instantiate your source separation model
wav2vec2_hidden_size = model.config.hidden_size
input_size = wav2vec2_hidden_size  # Input is the embedding of the mixture waveform
hidden_size = 256# Define the size of the hidden layer in your model
output_size = wav2vec2_hidden_size # Define the size of the output waveform
separation_model = SourceSeparationModel(input_size, hidden_size, output_size)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(separation_model.parameters(), lr=0.001)

# Assuming your train_loader is already defined
num_epochs = 2# Define the number of training epochs

train_loader = DataLoader(train_set, batch_size=2, shuffle=True, collate_fn=custom_collate_fn)

# Training loop
for epoch in range(num_epochs):
    for batch in train_loader:
        # Unpack the batch
        sampling_rate, mix_waveforms, source_waveforms_list = batch
        print(type(mix_waveforms))
        # Feature extraction using Wav2Vec 2.0 for the mixture
        mix_input_values = processor(mix_waveforms, return_tensors="pt", padding="longest").input_values
        mix_input_values = mix_input_values.squeeze()  # Remove unnecessary dimensions
        mix_outputs = model(mix_input_values.unsqueeze(0))  # Add batch dimension
        mix_logits = mix_outputs.logits.squeeze()

        # Feature extraction for each source waveform
        source_features_list = []
        for source_waveforms in source_waveforms_list:
            source_input_values = processor(source_waveforms, return_tensors="pt", padding="longest").input_values
            source_input_values = source_input_values.squeeze()  # Remove unnecessary dimensions
            source_outputs = model(source_input_values.unsqueeze(0))  # Add batch dimension
            source_logits = source_outputs.logits.squeeze()
            source_features_list.append(source_logits)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        separated_waveforms = separation_model(mix_logits)

        # Flatten the source features list for computing the loss
        flat_source_features = torch.cat(source_features_list, dim=0)

        # Compute the loss
        loss = criterion(separated_waveforms, flat_source_features)

        # Backward pass
        loss.backward()

        # Update weights
        optimizer.step()

    # Print training statistics (optional)
    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}')


Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


<class 'str'>


ValueError: type of mixture_waveforms unknown: <class 'str'>. Should be one of a python, numpy, pytorch or tensorflow object.

In [24]:
print(train_loader)